In [9]:
import numpy as np

def FK_PoE(q, a, rot, jt, M):
    T = POE(q, a, rot, jt)
    Tf = np.dot(T, M)
    R = Tf[:3, :3]
    p = Tf[:3, 3]
    return R, p, Tf

def POE(q, a, rot, jt):
    T = np.eye(4)
    n = len(q)
    for i in reversed(range(n)):
        if jt[i] == 'R':
            rot_hat = np.array([[0, -rot[i, 2], rot[i, 1]],
                                [rot[i, 2], 0, -rot[i, 0]],
                                [-rot[i, 1], rot[i, 0], 0]])
            e_rot_hat = np.eye(3) + rot_hat * np.sin(q[i]) + np.dot(rot_hat, rot_hat) * (1 - np.cos(q[i]))
        elif jt[i] == 'P':
            rot_hat = np.zeros((3, 3))
            e_rot_hat = np.eye(3)
        if jt[i] == 'R' and i > 0:
            Sv = -np.cross(rot[i, :], a[i, :]).T
        elif jt[i] == 'R' and i == 0:
            Sv = np.array([0, 0, 0])
        elif jt[i] == 'P':
            Sv = a[i, :].T
        p = (np.eye(3) * q[i] + (1 - np.cos(q[i])) * rot_hat + (q[i] - np.sin(q[i])) * np.dot(rot_hat, rot_hat)) @ Sv
        e_zai = np.vstack((np.hstack((e_rot_hat, p.reshape(3, 1))), [0, 0, 0, 1]))
        T = np.dot(e_zai, T)
    return T

# Define joint variables with specified values
t1 = 0
t2 = 0
t3 = -np.pi / 2
t4 = np.pi / 2
t5 = 0
t6 = 0
theta = np.array([t1, t2, t3, t4, t5, t6])

# Define other parameters
W1 = 109 / 1000
W2 = 82 / 1000
L1 = 425 / 1000
L2 = 392 / 1000
H1 = 89 / 1000
H2 = 95 / 1000

a = np.array([[0, 0, 0], [0, 0, H1], [-L1, 0, H1], [-L1 - L2, 0, H1], [-L1 - L2, -W1, 0], [-L1 - L2, 0, H1 - H2]])
rot = np.array([[0, 0, 1], [0, -1, 0], [0, -1, 0], [0, -1, 0], [0, 0, -1], [0, -1, 0]])
jt = 'RRRRRR'
M = np.array([[1, 0, 0, -L1 - L2], [0, 0, -1, -W1 - W2], [0, 1, 0, H1 - H2], [0, 0, 0, 1]])

# Calculate forward kinematics
R, p, Tf = FK_PoE(theta, a, rot, jt, M)

# Output the transformation matrix (pose of end-effector in base frame)
print("Transformation Matrix Tf:")
print(Tf)


Transformation Matrix Tf:
[[ 1.     0.     0.    -0.425]
 [ 0.     0.    -1.    -0.191]
 [ 0.     1.     0.     0.386]
 [ 0.     0.     0.     1.   ]]
